In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from getpass import getpass
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate
from langchain import hub
from langchain_anthropic import ChatAnthropic
from graphdatascience import GraphDataScience
import pandas as pd

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
anthropic_api_key = getpass()

 ········


In [9]:
openai_api_key = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [5]:
neo4j_uri = "neo4j+s://2fe3bf28.databases.neo4j.io"
neo4j_user = "neo4j"

In [6]:
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [7]:
def query_movies(query_string, query_type):
    queries = {
        "movie": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("movie_text_vectors", 50, v) YIELD node, score 
                    RETURN score, node.title AS title, node.overview AS overview
                    ORDER BY score DESC""",
        "theme": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("theme_vectors", 10, v) YIELD node, score 
                    MATCH (node)<-[:HAS_THEME]-(m)
                    RETURN node.description AS theme, score, m.title AS title, m.overview AS overview
                    ORDER BY score DESC
                    """,
        "theme_group": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("theme_group_summary_vectors", 3, v) YIELD node, score 
                    MATCH (node)<-[:IN_GROUP]-()<-[:HAS_THEME]-(m)
                    RETURN node.descriptions AS themes, score, m.title AS title, m.overview AS overview
                    ORDER BY score DESC"""
    }
        
    movie_df = gds.run_cypher(queries[query_type], {"queryString": query_string, "token":openai_api_key})
    return movie_df

In [75]:
theme_group_query = """
MATCH (node)<-[:IN_GROUP]-()<-[:HAS_THEME]-(m)
WITH m, max(score) AS score, collect(node.descriptions)[0] AS themes 
RETURN m.overview AS text, score, m{.title, themes:themes} AS metadata
ORDER BY score DESC 
LIMIT 50
"""

theme_query = """
MATCH (node)<-[:HAS_THEME]-(m)
WITH m, max(score) AS score 
RETURN m.overview AS text, score, m{.title, .url} AS metadata
ORDER BY score DESC
limit 50
"""

movie_query = """
MATCH (node)
RETURN node.overview AS text, score, node{.title, .url} AS metadata
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a movie expert providing information about movies.
                  Be as helpful as possible and return as much information as possible.
                  Do not answer any questions that do not relate to movies.
                  Provide your answer as a pipe-separated list of movie titles.
                  Only output the list of titles.
                  Do not inlcude any headings or other introductory words.
                  Answer any questions based solely on the context below:
                  <context>
                  {context}
                  </context>"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
])

movie_info_template= "overview: {page_content}, title: {title}"
document_prompt = PromptTemplate(input_variables=["page_content", "title"],
                                 template=movie_info_template
                                 )

combine_docs_chain = create_stuff_documents_chain(
    ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key), prompt, document_prompt=document_prompt)


def make_chain(query, index, k):
    vectorstore = Neo4jVector.from_existing_index(
        OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key),
        index_name=index,
        retrieval_query=query,
        url=neo4j_uri,
        username=neo4j_user,
        password=neo4j_password)
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    chain = create_retrieval_chain(retriever, combine_docs_chain)
    return chain

movie_chain = make_chain(movie_query, "movie_text_vectors", 50)
theme_chain = make_chain(theme_query, "theme_vectors", 25)
theme_group_chain = make_chain(theme_group_query, "theme_group_summary_vectors", 25)


In [76]:
def get_selected_movies(response, query, retriever):
    context = response['context']
    answers = [answer.strip() for answer in response['answer'].split("|") if answer.strip() != ""]
    movie_list = []
    for answer in answers:
        matched_context = [{"title":answer, "overview": doc.page_content}
                           for doc in context if doc.metadata['title'] == answer]
        if len(matched_context) == 1:
            movie_list.append(matched_context[0])
        else:
            movie_list.append({"title":answer, "overview": "Did not match context"})
    selected_movie_df = pd.DataFrame(movie_list)
    selected_movie_df['query'] = query
    selected_movie_df['retriever'] = retriever
    return selected_movie_df

def compare_searches(query):
    movie_result = movie_chain.invoke({"input": query})
    theme_result = theme_chain.invoke({"input": query})
    theme_group_result = theme_group_chain.invoke({"input": query})
    
    movie_selected = get_selected_movies(movie_result, query, "movie")
    theme_selected = get_selected_movies(theme_result, query, "theme")
    theme_group_selected = get_selected_movies(theme_group_result, query, "theme_group")
    selected_df = pd.concat([movie_selected, theme_selected, theme_group_selected])
    
    movie_set = set([document.metadata['title'] for document in movie_result['context']])
    theme_set = set([document.metadata['title'] for document in theme_result['context']])
    theme_group_set = set([document.metadata['title'] for document in theme_group_result['context']])

    movie_only = movie_set.difference(theme_set.union(theme_group_set))
    theme_only = theme_set.difference(movie_set.union(theme_group_set))
    theme_group_only = theme_group_set.difference(movie_set.union(theme_set))
    movie_theme = movie_set.intersection(theme_set).difference(theme_group_set)
    movie_theme_group = movie_set.intersection(theme_group_set).difference(theme_set)
    theme_theme_group = theme_set.intersection(theme_group_set).difference(movie_set)
    all_lists = movie_set.intersection(theme_set).intersection(theme_group_set)
    
    output_df = pd.DataFrame()
    output_df.loc["movie_answer", "result"] = movie_result['answer']
    output_df.loc["theme_answer", "result"] = theme_result['answer']
    output_df.loc["theme_group_answer", "result"] = theme_group_result['answer']
    output_df.loc["movie_only_context", "result"] = movie_only
    output_df.loc["theme_only_context", "result"] = theme_only
    output_df.loc["theme_group_only_context", "result"] = theme_group_only
    output_df.loc["movie_theme_context", "result"] = movie_theme
    output_df.loc["movie_theme_group_context", "result"] = movie_theme_group
    output_df.loc["theme_theme_group_context", "result"] = theme_theme_group
    output_df.loc["all_context", "result"] = all_lists
    
    return output_df, selected_df

In [77]:
pd.set_option('display.max_colwidth', None)

In [78]:
output_df, current_selected_df = compare_searches("What are some titles of movies about classical music?")
selected_df = current_selected_df
output_df

,result
movie_answer,| Beautiful Pleasant Trivial | Pioniere der Filmmusik - Europas Sound für Hollywood | I Hear a Symphony | Poster Child | Lenechka | Symphonic Sketches | Music Genres. Voices of the World | Asfalto che suona | Film Fest Gent | Milestones | The Musical Mind: A Portrait in Process | Fantasymphony II - A Concert of Fire and Magic | A Piano Tune | Nosferatu: A Symphony of Horror | Casanova and The Three Graces | The Battle for Swan Lake | Songs from the Hole | Doraemon the Movie: Nobita's Earth Symphony | The Piper | IN THE BASEMENT OF APOLLO HALL | Sergey Slonimsky. Bells | A Slippage in Five Movements | Cici's Sonata | Living Bach | Shchedrin Suite |
theme_answer,| Crescendo | Symphony n. 9 by Ludwig van Beethoven in St. Mark's Square | Symphonic Sketches | Als ik zelf de zon ben | Vika! | Angelheaded Hipster: The Songs of Marc Bolan & T. Rex | Pioniere der Filmmusik - Europas Sound für Hollywood | Beautiful Pleasant Trivial | Double Around the Interlude | Boléro |
theme_group_answer,|Als ik zelf de zon ben|Crescendo|Symphony n. 9 by Ludwig van Beethoven in St. Mark's Square|Symphonic Sketches|The Orchestra Chuck Built|Celtic Woman: 20th Anniversary Show|My Favorite Things: The Rodgers & Hammerstein 80th Anniversary Concert|Mozart's Requiem|The Only Girl in the Orchestra|Do You See Me?|Música!
movie_only_context,"{Reminder, Echo Sonata, The Whale Song, Lenechka, Istanbul Trilogy: Music, Andrey Konchalovsky’s Cinema Odyssey, Milestones, Pandora's Legacy, One For My Baby, Poster Child, A Love Letter to Cinema, Nocturne, Living Bach, Mūza, Symphony of the Holocaust, Was Once One, Film Fest Gent, In the Conservatory, The Nice Trip, Symphony of Suffering, Why Movies?, Cici's Sonata, Shchedrin Suite, The Battle for Swan Lake, Songs from the Hole, Casanova and The Three Graces, Sergey Slonimsky. Bells, A Slippage in Five Movements, Passing Time, The Movie Man, The Piper, Vampire Stories: Bluttanz, Planet Maximus, I. / II. / III., In The Shadow Of The Horns, IN THE BASEMENT OF APOLLO HALL}"
theme_only_context,"{The Tragic Death of a Frenchman, Oprah & The Color Purple Journey, Dicks: The Musical, Smosh: Anthony is Dead: The Funeral Roast, Angelheaded Hipster: The Songs of Marc Bolan & T. Rex, Maya! The Musical, Pretty Ugly - The Story Of The Lunachicks, Joker Out - Live from Arena Stožice, Dream Time, Mountainside, Portraits of a New York City Rock Band, Marta, la musical. Una ópera de Dani Umpi., Kikoriki are Making a Movie, Thriller 40, Show Tunes at the BBC, BACKSTAGE: The Movie, The Decline of Southwestern Civilization Pt. 1, The Great Curdling, Kevin Morby: Harlem River 10, Double Around the Interlude, TAYLOR SWIFT | THE ERAS TOUR, Trouble with Johnny, Five Percent, Be Our Guest! Behind the Scenes of Beauty and the Beast: The Musical, Big Night of Musicals by the National Lottery - 2024, Baltimore, Blockbuster, BLACKPINK WORLD TOUR [BORN PINK] FINALE IN SEOUL, Vergeet Barbara - De Musical, A Minor Problem, Maggie Lindemann: SUCKERPUNCH WORLD TOUR, Otto Baxter: Not A F***ing Horror Story, I Love Nancy Meyers, Absolve, Vika!, Cruel Like You, Armin van Buuren Presents This is Me: Feel Again}"
theme_group_only_context,"{Do You See Me?, My Name's Ben Folds – I Play Piano, The Runaway, Fanny: The Other Mendelssohn, Ryuichi Sakamoto | Opus, American Symphony, Celtic Woman: 20th Anniversary Show, What If?, Wait and See, Das Märchen von der Zauberflöte, Brilliant A, Dao, Pho Va Piano, Between the Temples, The Devil Went Down To Islington, Carnaval: La Fiesta entre la Carne y el Espíritu, Piano in the Bushes, Mozart's Requiem, The Ukeman, The Orchestra Chuck Built, My Favorite Things: The Rodgers & Hammerstein 80th Anniversary Concert, Bee's Knee's, Trolls Band Together, Keys to the Heart, Benevolence, The Complete Taylor Swift Story, Stars, Eye for an Eye, Lil' Santa's Book Club: The Christmas Reindeer, The Only Girl in the Orchestra, Roots of Fire, Goddess of Speed, Christmas at Graceland, Le Nozze Di Figaro - Opera Bal

In [79]:
output_df, current_selected_df = compare_searches("What are some titles of movies about baseball?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,| Young Again | Thunder Rolls! The World of Blind Baseball | Shohei Ohtani: Beyond the Dream | Monster Under The River | The Astros Edge | Indoor Baseball | unDEFILED | Below the Rim |
theme_answer,"|Carson Bigbee: The Pirate of America's Pastime|unDEFILED|Monster Under The River|Heritage Minutes: Mary ""Bonnie"" Baker|Samurai Japan: The Story Behind the WBC Clean Sweep|The Astros Edge|Shohei Ohtani: Beyond the Dream|Young Again|Thunder Rolls! The World of Blind Baseball"
theme_group_answer,"| Samurai Japan: The Story Behind the WBC Clean Sweep | Carson Bigbee: The Pirate of America's Pastime | unDEFILED | Monster Under The River | Heritage Minutes: Mary ""Bonnie"" Baker | Smashing Pumpkins | The Astros Edge | Shame |"
movie_only_context,"{Butchy's Big Pitch, Groundbreakers, Sole: The Movie, I'll Crush Y'all, A Movie With No Views On Letterboxd, How India Stole and Killed Cricket, Humbly Super Famous, Swing Bout, Holiday Heist, Made in Massachusetts: 100 Years of Filmmaking in the Bay State, Fuggeddaboutitt, Brothers, Heads and Tails, Below the Rim, The Godfrey Project, Ethan the Roid Junkie, The Big Catch, Squatch Junkies, Frank Capra: Mr. America, Blue Star, Why Movies?, Diamante – Fussballgott, Falling in Love Like in Movies, The Power of the Strike, No Ball Games, Because They Believed, Movie (Choke), The Movie Man, Labranza, Joe, Left Brain, Blood Money, BRADMAN: The Movie, Bowlhead, The Hurler: A Campion's Tale}"
theme_only_context,"{Adventures in Perception, We Joined A Cult, The Tragic Death of a Frenchman, Oprah & The Color Purple Journey, Swimming in a Sand Pool, Strade perdute - Filmmaker 23, Sharko, Živko Nikolić - Unfulfilled Dream, La stoccata vincente, Mountainside, Mi Ópera Prima, Trouble with Johnny, Love & Films, Merchant Ivory, MARN, dressed in nudity, Three Solar Exercises, Dans le Film, le Film des Films au Cinema Pour le Cinema qui Joue des Films, Consumed, Singapore Films: To The World & Back}"
theme_group_only_context,"{Charles Barkley Comes Out, Wardell Stephen Curry, Young Guns: The New Tennis Titans, Kingdome, Warrior Strong, Out of Bounds, Al Chile!, Backboard, NFL Films - The History of Player Introductions, The Netflix Slam, Pickup, Djokovic Unmasked, The Minister of Defense, After Dawn, Terrion Arnold: Prolific, One More Shot, The Ticket, Hada, Hard Feelings, Triangle Park, La cancha, 5th QTR, Leo, Bye Bye Barry, Dealova, Rally, Big Time, Sue Bird: In the Clutch, Divine Providence, Eureka, Hoop Cities - NBA Feature Documentary, Home and Away, Giannis: The Marvelous Journey, Imbroda, el legado del maestro, Rudy Gobert N°27}"
movie_theme_context,"{Young Again, Thunder Rolls! The World of Blind Baseball, Shohei Ohtani: Beyond the Dream, Kanashimi, Indoor Baseball}"
movie_theme_group_context,"{Iceman: A George Gervin Story, Lady Ballers, GROUNDED., Freaky Tales, Gold Blooded - NBA Feature Documentary}"
theme_theme_group_context,"{The Hoop Dreams Holiday Special, Heritage Minutes: Mary ""Bonnie"" Baker, Smashing Pumpkins, Shame, The Many Worlds of George Goodman}"
all_context,"{Carson Bigbee: The Pirate of America's Pastime, Monster Under The River, Samurai Japan: The Story Behind the WBC Clean Sweep, unDEFILED, The Astros Edge}"


In [80]:
output_df, current_selected_df = compare_searches("What are some movie titles about freedom?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,|Liberty|Naked Nations|American Outlaws|FOUR DAYS FOR FREEDOM: NAPLES 1943|If Everyone Just Leaves|Celluloid Underground|Poor Things|Silent Cries|Here and Nowhere|Nobody Leaves Alive|Fly Free|My Sister's Keeper|Letters To Freedom|Infinity!|Waterbird|In Freedom
theme_answer,"|FREEDOM MAN: THE BALLAD OF THE BRICK|1521: The Quest for Love and Freedom|It's A Gray Gray World!|Yesterday|Sedin Kuyasha Chilo|Shyamchi Aai|Sabittri|The Ticket|Breaking the Ice|The History of Indian Elections|4 Walls|Notes on a Summer|The Prince of Egypt: The Musical|Hunting Party|1989: Chceme dýchat|My Freedom|Rise|Transformation|Blue|Defend Your Name|Metal Belt|Amanita|Night in Newark|Getting Fired From Office|Zindagi Zindabaad|Fille Fleur|Water Baby|Code Break|Breath|Feet Fear Freedom|Not Born Yesterday|Gendelyk|Free Beats: The Musical Journey of CHEN Ming Chang|Ukrainian Identity|Our Land, Our Freedom"
theme_group_answer,|Bad Romance: The Vicky White Story|Two Balloons|Freedom|Metal Belt|My Freedom|Rise|4 Walls|Defend Your Name|The Prisoner of Wakhan|Zindagi Zindabaad|Water Baby|1989: Chceme dýchat|Breath|Nyanga|Flowing - My Dream of More Freedom|The Last Thread
movie_only_context,"{If Everyone Just Leaves, Kanaani, Poor Things, PRODUCT, Communist Osman, Fly Free, The Busy World Around Me., Che, In Freedom, Exit, Winner, A Long Shot, Exile, Landless, Naked Nations – Tribe Hong Kong, Letters To Freedom, The Conqueror (Hollywood Fallout), Love of the Soviet Union, When We Return, Lines of Resistence, Silent Cries, Happy Places, Air Force Two, Celluloid Underground, From 8 Movies To 1, I Believe in America, Cine Libre, Waterbird, The Godfrey Project, American Outlaws, The Nice Trip, Infinity!, Trailer of a Film That Will Never Exist: Phony Wars, My Sister's Keeper, Why Movies?, Little Death, Nobody Leaves Alive, Liberty, Fly Hard, Memories, Prisoners, Land of My Dreams, The Movie Man, Movie (Choke), Showdown at the Grand, Kannagi, Here and Nowhere, Under a Blue Sun, What is the Point?}"
theme_only_context,"{Sabittri, FREEDOM MAN: THE BALLAD OF THE BRICK, It's A Gray Gray World!, 1521: The Quest for Love and Freedom, Shyamchi Aai, The Ticket, Yesterday, Sedin Kuyasha Chilo}"
theme_group_only_context,"{El Grito De Emma, Ex You, No Way Through, The Last Thread, Flowing - My Dream of More Freedom, Nyanga, Boots On The Ground, The Strike}"
movie_theme_context,{}
movie_theme_group_context,{}
theme_theme_group_context,"{Water Baby, Hunting Party, Feather Family, Metal Belt, Zindagi Zindabaad, Blue, Free Beats: The Musical Journey of CHEN Ming Chang, The Prince of Egypt: The Musical, Fille Fleur, Bad Romance: The Vicky White Story, Girl, Not Born Yesterday, Dealer, Gendelyk, Sorbeltza, Rise, Psychedelic Revolution: The Secret History of the LSD Trade, My Freedom, Paradise Is Burning, Amanita, Two Balloons, Stamp Our Story, Ukrainian Identity, Rencores, Notes on a Summer, The Prisoner of Wakhan, Feet Fear Freedom, Code Break, Ado Japan Tour 2023 ""MARS"", 4 Walls, Defend Your Name, Breaking the Ice, Night in Newark, 1989: Chceme dýchat, Getting Fired From Office, Breath, Transformation, The History of Indian Elections, Our Land, Our Freedom, Decisions Decisions, Bloc Party: So Here We Are}"
all_context,{FOUR DAYS FOR FREEDOM: NAPLES 1943}


In [81]:
output_df, current_selected_df = compare_searches("What are some titles of dark comedy movies?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"|Night|The Beast of Comedy|Silence|The Darker The Shadow, The Brighter The Light|My Movie|Et Tu|Movie (Choke)|The Butler|Affirmations|Ab Absurdo|DarkGame|What Happens in the Dark|Elegy - Director's Cut|End All, Say All|Her Dark Reflection|The Day Is Long and Dark|Brave the Dark|Death Perception|Little Death|Stitches|In The Shadow Of The Horns|The Electric Fleapit|Accessory|The Critic|mother, son, Comedy!|Dindin|Stand-Up - My Life Is A Joke|To the Brink|Power Cut|Død af grin|Dead Serious|For When You Get Lost|Into the Darkness|Murder In The Theater|Date Night|Darkling|Pattaya Explicit|In Search Of...Detective Barnaby And His Descent Into The Haunted Mansion|Holiday Heist|The Martini Shot|Egun|GrimTrax|Kill The Lights|MrMrMILKMAN in Donnie Darko|What's Behind You|The Roommate"
theme_answer,"| Funny Not Funny | Sleepyhead | Død af grin | Natural Causes | Fishmonger | Do Not Expect Too Much from the End of the World | Mole | Klokkenluider | Death Perception | Accessory | Triple Oh! | Deal of a Lifetime | The Year of Staring at Noses | World's Greatest Dad | Stand-Up - My Life Is A Joke | Mike Ward : Noir | Dindin | Sheryl | Eppura | Barely Breathing | Irish Catholic | End All, Say All | Silence | Affirmations | Anemone: A Fairy Tale for No Kids | Double Trouble | Thiraiyin Marupakkam | Abomination | Sam Jay: Salute Me or Shoot Me | Sweet Dreams | Otto Baxter: Not A F***ing Horror Story | Should Be Long Gone | Holiday Heist | Au Revoir, Slumdogz | The Emu War | The Choice Has Been Made | Void | The Moon Is Upside Down | Home Town Down Time | The Treasure of the Cyclops | The Puppet Asylum | Cannibal Mukbang | Ghost Babe | Dare! | The Haunted Baby Carriage From Hell | Shaky Shivers | Paranormal Drunktivity | In Search Of...Detective Barnaby And His Descent Into The Haunted Mansion | The Happy Hotel | Red Light District Shark Attack"
theme_group_answer,"| Dindin | Stand-Up - My Life Is A Joke | World's Greatest Dad | Silence | Fishmonger | End All, Say All | Barely Breathing | Klokkenluider | Mole | Død af grin | Mike Ward : Noir | Do Not Expect Too Much from the End of the World | Natural Causes | Triple Oh! | Accessory | The Choice Has Been Made | Funny Not Funny | Deal of a Lifetime | Sheryl | Home Town Down Time | The Moon Is Upside Down | Death Perception | Irish Catholic | Void | The Year of Staring at Noses | Affirmations | Eppura | With Love, from AIDAN | I'm Not a Robot | Lessons of Tolerance | Los | The End of Voyage | Sleepyhead | Double Trouble | Thiraiyin Marupakkam | Abomination | Sam Jay: Salute Me or Shoot Me | Anemone: A Fairy Tale for No Kids | Sweet Dreams | Au Revoir, Slumdogz | Holiday Heist | Should Be Long Gone | The Haunted Baby Carriage From Hell | Shaky Shivers | Paranormal Drunktivity | In Search Of...Detective Barnaby And His Descent Into The Haunted Mansion | The Happy Hotel | Red Light District Shark Attack | Honto ni Atta! Noroi no Video 103 | Otto Baxter: Not A F***ing Horror Story"
movie_only_context,"{What the Blood Moon Brings: Messiah of Evil, A New American Nightmare, Night, The Roommate, The Critic, The Terror of Tarheel State: Making The Dark Power, Kill The Lights, GrimTrax, Brave the Dark, Tastes of Horror, mother, son, Comedy!, Et Tu, Date Night, The Electric Fleapit, DarkGame, What Happens in the Dark, Murder In The Theater, The Day Is Long and Dark, The Martini Shot, The Darker The Shadow, The Brighter The Light, Into the Darkness, Ab Absurdo, Little Death, Egun, Stitches, The Butler, The Beast of Comedy, My Movie, Dead Serious, Elegy - Director's Cut, To the Brink, MrMrMILKMAN in Donnie Darko, American Comedy, Movie (Choke), Her Dark Reflection, Power Cut, What's Behind You, Pattaya Explicit, Darkling, For When You Get Lost, In The Shadow Of The Horns}"
theme_only_context,"{The Emu War, Dare!, Ghost Babe, The Treasure of the Cyclops, The Puppet Asylum, Cannibal Mukbang}"
theme_group_only_context,"{Lessons of Tolerance, Los, I'm Not a Robot, The End of Voyage, With Lov

In [82]:
output_df, current_selected_df = compare_searches("What are some movies about artificial intelligence?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"| So Unreal | Artificial Intelligence, 2023 | The AI Report | The Diary of Sisyphus | Synthesis | Original Short Film | The Conscious ""I"" | Mad Scientists: A.I. Maxima | The Creator | AI History 1890-2090 | Illusions of Movement | With Love, from AIDAN | William Quail's Pyramid | Fair Weather Friends | A.I. Perversion | Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare | To Live and Die in AI | Blue Fire | Primacy | Ilya | AI Lobotomy | 512X512 | Double iSmart | Cyborg Society | Love Machina | Human or AI? The future of beauty standards | 4°C | EDDY BAIR | T-130 | The Perfect Human | Telephones 4 Eyes | The Electric Kiss | Literally No Place | Sci-fi Silence | Two Gracious Uncles Smooched to the Beat | To The Future: The Movie | Deep Fake Love | The Beast | Teri Baaton Mein Aisa Uljha Jiya | The Vivid Unknown | AIMEE: The Visitor | A.I. Made Me Do It: Ophelia | Cosmic Miniatures | Storyteller | H-151: Perfect Machine | Aire: Just Breathe | Intercept: A Century of Signals Intelligence | Death Cleaning | Impossible Decision"
theme_answer,"| CAPTCHA | Primacy | You Are Human, and You Are Lover | 3 Dialogues About the Future | The Female Side Effect | Pesudo | Landless | The Creator | CCC | Illusions of Movement | Love Me | The Beast | Lady | So Unreal | Cyborg Society | A.I. Perversion | Two Gracious Uncles Smooched to the Beat | Lights Over Montgomery County | H.O.M.E. | Blue Fire | The Conscious ""I"" | Ilya | Love Machina | Sucker | Mad Scientists: A.I. Maxima | JUST DESSERTS | Original Short Film |"
theme_group_answer,"| Pesudo | The Perfect Human | CAPTCHA | i ai | Love Me | So Unreal | The Female Side Effect | Mad Scientists: A.I. Maxima | Lady | Cyborg Society | 3 Dialogues About the Future | The Beast | Primacy | CCC | You Are Human, and You Are Lover | Illusions of Movement | The Creator | Ilya | Landless | The Electric Kiss | Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare | Smog | EDDY BAIR | Synthesis | Whisper of a Butterfly | A.I. Made Me Do It: Ophelia | Agents | Exit Pangea | Artificial Intelligence, 2023 | 4°C | Fair Weather Friends | AIMEE: The Visitor | AI History 1890-2090 | To Live and Die in AI | Dotty and the Tower of Eternity | The AI Report | A Tree Once Grew Here | Ghost in the Shell: SAC_2045 The Last Human | Voice in My Head | With Love, from AIDAN | Cosmic Miniatures | Aire: Just Breathe | MOVE | Literally No Place | Human or AI? The future of beauty standards | Storyteller | AI Lobotomy | H.O.M.E. | Blue Fire | JUST DESSERTS"
movie_only_context,"{The Vivid Unknown, The Diary of Sisyphus, To The Future: The Movie, Telephones 4 Eyes, H-151: Perfect Machine, Intercept: A Century of Signals Intelligence, Double iSmart, William Quail’s Pyramid, Impossible Decision, Death Cleaning, Deep Fake Love, T-130}"
theme_only_context,"{The Tragic Death of a Frenchman, FU*AB, Lights Over Montgomery County, Head Scientist, Dead Again, Night Run, Ultraseven IF Story: The Future 55 Years Ago, Sucker, Bell Jingler 2023, Mountainside, Cybergrime, Anomaly, Kikoriki are Making a Movie, Postcard From Earth, The Olaria Chronicles, Doctor Who: 60 Years of Secrets & Scandals, Cirkuit Breac, Eternal return / op.1 wonderful world, AI Generated These I Had No Hands}"
theme_group_only_context,"{Exit Pangea, Ghost in the Shell: SAC_2045 The Last Human, Agents, MOVE, A Tree Once Grew Here, Smog, Voice in My Head, Whisper of a Butterfly, Dotty and the Tower of Eternity}"
movie_theme_context,"{Love Machina, Teri Baaton Mein Aisa Uljha Jiya, A.I. Perversion, Original Short Film, Sci-fi Silence, 512X512, Two Gracious Uncles Smooched to the Beat, The Conscious ""I""}"
movie_theme_group_context,"{Synthesis, Artificial Intelligence, 2023, Aire: Just Breathe, Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare, 4°C, Fair Weather Friends, Storyteller, AIMEE: The Visitor, The Electric Kiss, EDDY BAIR, A.I. Made Me Do It: Ophelia, Human or AI? The future of beauty standards, With

In [83]:
output_df, current_selected_df = compare_searches("What are some movies about growing up in a small town?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your rate limit (https://docs.anthropic.com/claude/reference/rate-limits). Please reduce the the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [ ]:
output_df, current_selected_df = compare_searches("What are some movies about dogs?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

In [ ]:
output_df, current_selected_df = compare_searches("What are some movies about Europe in the 1960s?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

In [ ]:
output_df, current_selected_df = compare_searches("What are some movies set in the Amazon rain forest?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

In [ ]:
output_df, current_selected_df = compare_searches("What are some movies about the president of the United States?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

In [ ]:
selected_df

In [ ]:
selected_df.to_csv("movie_answers.csv", index=False, header=True)

In [57]:
pd.set_option('display.max_colwidth', None)

In [8]:
movie_query = """
MATCH (node)
RETURN node.overview AS text, score, node{.title, .tagline} AS metadata
"""

movie_vectorstore = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key),
    index_name="movie_text_vectors",
    retrieval_query=movie_query,
    url=neo4j_uri,
    username=neo4j_user,
    password=neo4j_password
)

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a movie expert providing information about movies.
                  Be as helpful as possible and return as much information as possible.
                  Do not answer any questions that do not relate to movies.
                  Always include the title of the movie in your answer.
                  Answer any questions based solely on the context below:
                  <context>
                  {context}
                  </context>"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
])

In [10]:
movie_info_template= "overview: {page_content}, title: {title}"
document_prompt = PromptTemplate(input_variables=["page_content", "title"],
                                 template=movie_info_template
                                 )


In [35]:
theme_group_retriever = theme_group_vectorstore.as_retriever(search_kwargs={"k": 5})
theme_retriever = theme_vectorstore.as_retriever(search_kwargs={"k":15})
movie_retriever = movie_vectorstore.as_retriever(search_kwargs={"k": 30})

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(
    ChatOpenAI(temperature=0, openai_api_key=openai_api_key), prompt, document_prompt=document_prompt

)
theme_group_chain = create_retrieval_chain(theme_group_retriever, combine_docs_chain)
theme_chain = create_retrieval_chain(theme_retriever, combine_docs_chain)
movie_chain = create_retrieval_chain(movie_retriever, combine_docs_chain)

In [40]:
theme_group_retriever.get_relevant_documents("sharks")

[Document(page_content='The curse of Count Dracula lives on in shark-infested waters, claiming the lives of a tourist community. A sea hunt for the new species results in monsters, madness and bloodshed. This great white is putting the bite back into terror, and it has help with the aid of new vampires intent on seeing it survive.', metadata={'title': 'Sharkula'}),
 Document(page_content="Camille cultivates flowers on 7000 meters square in Aveyron, France. During all year, « le Jardin de Veillac » structures itself : building a new greenhouse, preparing seedlings, collecting flowers, preparing bouquets and, finally, selling them on Rodez's marketplace, where flowers unveil their beauty and spread in the city.", metadata={'title': 'La Ferme florale de Camille'}),
 Document(page_content='During the seed harvest season in Ban Rabue Village, Ngern is responsible for collecting and storing seeds for the upcoming year. Unfortunately, a group of black-faced thieves attempt to steal the seeds,

In [36]:
theme_group_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental technique, t

In [31]:
theme_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental technique, t

In [18]:
movie_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="The ultimate deep dive into the world of shark cinema: filmmakers, critics, scholars and conservationists explore the weird, wild cinematic legacy of sharks on film and audiences' undying fascination with these misunderstood creatures.", metadata={'title': 'Sharksploitation'}),
  Document(page_content='In a not so distant future, great white sharks have gone extinct. Over fishing, climate change, pollution have all contributed to the species complete annihilation from the seas... but, land may be their new hunting ground! When a mad scientist plays god and genetically engineers a shark/human, he unleashes the highly viral monster on the unsuspecting citizens of a nearby small town. Now, a group of friends must band together if they are to survive the “sharkocalypse”. Run, hide, fight... or be turned into CHUM!', metadata={'title': 'CHUM!', 'tagline': 'You’re Gonna Need A Better Vaccine'}),
  D

In [37]:
theme_group_chain.invoke({"input": "Find as many movies as you can about high school."})

{'input': 'Find as many movies as you can about high school.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental techniq

In [20]:
summary_chain.invoke({"input": "Find as many movies as you can about high school."})

{'input': 'Find as many movies as you can about high school.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental techniq